In [ ]:
# Import WebSocket client library
from websocket import create_connection
import threading
import json
import pandas as pd
import numpy as np
import websocket
import datetime
from functools import partial
import timeit
import requests
import time
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
class KrakenWebSocketClient:
    """
    Kraken Client Interface

    """

    def __init__(self, data_class ):
        websocket.enableTrace(False)
        self.data_class=data_class
        ws = websocket.WebSocketApp("wss://ws.kraken.com/",
                                         on_message=self.on_message,
                                         on_error=self.on_error,
                                         on_close=self.on_close)
        self.ws = ws
        self.ws.on_open = self.on_open
        self.ws.run_forever()

    def on_message(self,ws, message):
        if "event" not in message: 
            formated_point = json.loads(message)
#             print('WebSocket: '+formated_point[1][0]+' '+ formated_point[1][1]+' '+formated_point[1][2]+' '+str(formated_point[1][-1]))
            self.data_class.to_data_que(formated_point[1])
            
        return message

    def on_error(self,ws, error):
        print(error)
        return error

    def on_close(self, a,b,c):
        self.ws.close()
        print("### closed ###")   
        # this is where the final data will be pushed to the dbs or something 

    def run(self, *args):
        global driver
        driver = True
        while driver:
            try:
                time.sleep(1)
#                 I think we can use the below part to stop the code 
#                 p = input()
#                 self.ws.send(p)
            except KeyboardInterrupt:
                driver = False
        time.sleep(1)
        self.ws.close()
        print("thread terminating...")

    def on_open(self,ws):
        self.ws.send('{"event":"subscribe", "subscription":{"name":"ohlc"}, "pair":["XBT/USD"]}')
        t = threading.Thread(target=self.run)
        t.start()
        



In [ ]:
class BTC_Data(): 
    def __init__(self):
        self.data_que=[]
        self.one_min =[]
        
    def to_data_que(self, data_point: list):
        if len(self.data_que)==0:                      # Empty que just add the point 
            self.data_que.append(data_point)

        else: 
            if data_point[1] == self.data_que[-1][1]: # In the Same interval: add the point
                self.data_que.append(data_point)
                
            else: 
                # pop the last point                 
                # Add to the 1 min data frame 
                new_point = self.data_que.pop()
                self.to_one_min(new_point)
                # clear the que 
                # add the new point 
                self.data_que.clear()
                self.data_que.append(data_point)
                
                
    def to_one_min(self, data_point:list):
        formated_point = [
            int(float(data_point[1])),
            datetime.datetime.utcfromtimestamp(int(float(data_point[1]))),
            'BTCUSD',
            float(data_point[2]),
            float(data_point[3]),
            float(data_point[4]),
            float(data_point[5]),
            float(data_point[7])    
        ]
        self.one_min.append(formated_point)
        if len(self.one_min) == 3: 
            print('1 min list')
            for x in self.one_min:
                print(x)
        
        
# 0 time	decimal	Begin time of interval, in seconds since epoch
# 1 etime	decimal	End time of interval, in seconds since epoch
# 2 open	decimal	Open price of interval
# 3 high	decimal	High price within interval
# 4 low	decimal	Low price within interval
# 5 close	decimal	Close price of interval
# 6 vwap	decimal	Volume weighted average price within interval
# 7 volume	decimal	Accumulated volume within interval
# 8 count

#     unix	date	symbol	open	high	low	close	Volume BTC
        
    


In [ ]:
# USE THIS FOR TESTING IN JUYPTER NOTEBOOKS. USE THE BELOW CELL IN FOR .PY VCODE PART ONCE WE MERGE IT ALL
BTC =BTC_Data()
kraken_client = KrakenWebSocketClient(BTC)

In [ ]:
def main():
#     Before load
    BTC =BTC_Data()
    kraken_client = KrakenWebSocketClient(BTC)


# if __name__ == '__main__':
#     main()

In [ ]:
# Everything above this is working   # below is a work in progress

In [ ]:
########## START of SQL stuff ##########
# Load ENV Files for SQL database
env_vars = {} # or dict {}
with open('src/env.txt') as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        key, value = line.strip().split('=', 1)
        env_vars[key]= value
print(env_vars)

# create sqlalchemy engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user=env_vars['USER'],
                               pw=env_vars['PASSWORD'],
                               host=env_vars['HOST'],
                               db=env_vars['DB']))
########## END of SQL stuff ##########

# pull timeframe data
# df_1min = pd.read_sql_table('BTC_Ticker_Data', con=engine)
# df_1min.to_csv('BTC_Ticker_Data_1_Min.csv')
# print('1 Min Pulled in...')
# df_5min = pd.read_sql_table('BTC_Ticker_Data_5_Min', con=engine)
# df_5min.to_csv('BTC_Ticker_Data_5_Min.csv')
# print('5 Min Pulled in...')
# df_30min = pd.read_sql_table('BTC_Ticker_Data_30_Min', con=engine)
# df_30min.to_csv('BTC_Ticker_Data_30_Min.csv')
# print('30 Min Pulled in...')
# df_1hr = pd.read_sql_table('BTC_Ticker_Data_1_Hour', con=engine)
# df_1hr.to_csv('BTC_Ticker_Data_1_Hour.csv')
# print('1 hour Pulled in...')
# df_4hr = pd.read_sql_table('BTC_Ticker_Data_4_Hour', con=engine)
# df_4hr.to_csv('BTC_Ticker_Data_4_Hour.csv')
# print('4 hour Pulled in...')
# df_12hr = pd.read_sql_table('BTC_Ticker_Data_12_Hour', con=engine)
# df_12hr.to_csv('BTC_Ticker_Data_12_Hour.csv')
# print('12 hour Pulled in...')
# df_24hr = pd.read_sql_table('BTC_Ticker_Data_24_Hour', con=engine)
# df_24hr.to_csv('BTC_Ticker_Data_24_Hour.csv')
# print('24 hour Pulled in...')

df_1min  = pd.read_csv('BTC_Ticker_Data_1_Min.csv') 
print('1 Min Pulled in...')
df_5min  = pd.read_csv('BTC_Ticker_Data_5_Min.csv') 
print('5 Min Pulled in...')
df_30min = pd.read_csv('BTC_Ticker_Data_30_Min.csv') 
print('30 Min Pulled in...')
df_1hr   = pd.read_csv('BTC_Ticker_Data_1_Hour.csv') 
print('1 hour Pulled in...')
df_4hr   = pd.read_csv('BTC_Ticker_Data_4_Hour.csv') 
print('4 hour Pulled in...')
df_12hr  = pd.read_csv('BTC_Ticker_Data_12_Hour.csv') 
print('12 hour Pulled in...')
df_24hr  = pd.read_csv('BTC_Ticker_Data_24_Hour.csv')
print('24 hour Pulled in...')

# create a loop to deal with extra columns for now this works fine
df_1min  = df_1min.iloc[: , 2:]
df_5min  = df_5min.iloc[: , 2:]
df_30min = df_30min.iloc[: , 2:]
df_1hr   = df_1hr.iloc[: , 2:]
df_4hr   = df_4hr.iloc[: , 2:]
df_12hr  = df_12hr.iloc[: , 2:]
df_24hr  = df_24hr.iloc[: , 2:]

In [ ]:
## Pulling in gap data from last point in df

In [ ]:
df_1min.iloc[-1]['Unix Timestamp'] # gets the last data point 

In [ ]:
# used to get the trade data but only returns last 1000 points of data from the date given 
def nano_after_epoch(t):
    """Converts DateTime object to int nanoseconds after epoch"""
    return int(t.replace(tzinfo=datetime.timezone.utc).timestamp() * 1000000000)

start_time = (df_1min.iloc[-1]['Unix Timestamp']+60) * 1000000000 # add 60 to start at first missing point
end_time = nano_after_epoch(datetime.datetime.utcnow())
dfList = []

print('unix start: ',start_time,'unix End: ',end_time)

start = timeit.timeit() #  for timing 
while start_time < end_time :
    print(f"Retrieving data from {datetime.datetime.utcfromtimestamp(start_time // 1000000000)}...")

    query_string = 'https://api.kraken.com/0/public/Trades?pair=xbtusd&since='+ str(start_time)
    resp = requests.get(query_string).json()

    # Append new data to dataframe
    tempFrame = pd.DataFrame(resp['result']['XXBTZUSD'])
    dfList.append(tempFrame)
    
    # Determine last retrieved date
    start_time = int(resp['result']['last'])

    # Sleep to avoid being rate limited
    time.sleep(1.8)

end = timeit.timeit()
print('elapsed time: ',end - start)

In [ ]:
# combine to Panadas DF 
trade_data = pd.concat(dfList, ignore_index=True)
trade_data

In [ ]:
# loop through creating 1 min data frames 

# [<price>, <volume>, <time>, <buy/sell>, <market/limit>, <miscellaneous>]
trade_data = trade_data.astype({0: float, 1: float, 2:float})

############# TODO FIX THIS WHEN COPYING 
start_time = 1634429100 # Delete this line and use the start date from before 
end_time = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc).timestamp() 
# use the same end time that was used in the call to api get trades data NOT like i did here
############# 

new_one_min_data =[]
symbol_val   = 'BTCUSD'
open_val, high_val, low_val, close_val, volume_val = 0,0,0,0,0
while start_time < end_time:  
    relevant_rows = trade_data[
                (trade_data[2] >= start_time) &
                (trade_data[2] < (start_time + 60) )
                ]
    unix_val     = start_time
    datetime_val = datetime.datetime.utcfromtimestamp(start_time)
    if len(relevant_rows) !=0: 
        open_val     = relevant_rows.iloc[0][0]
        close_val    = relevant_rows.iloc[-1][0]
        high_val     = np.max(relevant_rows[0])
        low_val      = np.min(relevant_rows[0])
        volume_val   = round(relevant_rows[1].sum(), 8)   
    else :# If there are no trades in the given time frame
        open_val     = new_one_min_data[-1][6]
        close_val    = new_one_min_data[-1][6]
        high_val     = new_one_min_data[-1][6]
        low_val      = new_one_min_data[-1][6]
        volume_val   = 0 
        
    # Creates 1 min df 
    new_row = [unix_val, datetime_val, symbol_val, open_val, high_val, low_val, close_val, volume_val]
    new_one_min_data.append(new_row)
    # add 1 min to start time 
    start_time+=60
    
    ############################## 
    if (len(new_one_min_data)%1440 ==0): # used for logging can be deleted in main program 
        print(datetime_val)

In [ ]:
new_one_min_data_DF = pd.DataFrame(new_one_min_data,columns=df_1min.columns)

# THIS LINE BELOW IS ONLY BC I MESSED UP USING THE WRONG END TIME BEFORE.  
# USED TO REMOVE THE EMPTY TIME FRAMES AT THE END OF THE DF
new_one_min_data_DF = new_one_min_data_DF.iloc[0:80371] 

In [ ]:
# add to old data 
new_df_1min = pd.concat([df_1min,new_one_min_data_DF], ignore_index=True)

In [198]:
def combine_time_frames(data, first_date, increment, dfString):
    new_data =[]
    symbol_val   = 'BTCUSD'
    open_val, high_val, low_val, close_val, volume_val = 0,0,0,0,0
    start_time = first_date + 60
    while True: 
        relevant_rows = data[
                    (data['Unix Timestamp'] >= start_time) &
                    (data['Unix Timestamp'] < (start_time + (60*increment)) )
                    ]
        if len(relevant_rows)!=increment:
            break
        # loop break condition if relevant rows does not have the right amount of rows in it 
        
        unix_val     = relevant_rows.iloc[-1]['Unix Timestamp']
        datetime_val = relevant_rows.iloc[-1]['Date']
        open_val     = relevant_rows.iloc[0]['Open']
        high_val     = relevant_rows['High'].max()
        low_val      = relevant_rows['Low'].min()
        close_val    = relevant_rows.iloc[-1]['Close']
        volume_val   = relevant_rows['Volume'].sum()
        
        # Creates new row
        new_row = [unix_val, datetime_val, symbol_val, open_val, high_val, low_val, close_val, volume_val, dfString]
        new_data.append(new_row)
        
        # add 1 min * increment to start time 
        start_time+=(60 *increment)
        
    return new_data


In [199]:
# create higher demensional frames 

# df_1min  ######################### 
    # Already done above 
    # Move code here 

cols = ['Unix Timestamp','Date','Symbol','Open','High','Low','Close','Volume','Time Frame']
# df_5min  #########################
last_5min = df_5min.loc[len(df_5min)-1,'Unix Timestamp']
rows_to_add = combine_time_frames(new_df_1min, last_5min, 5, "5 Min")
rows_to_add_df = pd.DataFrame(rows_to_add,columns=cols)
new_df_5min = pd.concat([df_5min, rows_to_add_df], ignore_index=True)
print('5 done')
# df_30min #########################
last_30min =df_30min.loc[len(df_30min)-1,'Unix Timestamp']
rows_to_add = combine_time_frames(new_df_1min, last_30min, 30, "30 Min")
rows_to_add_df = pd.DataFrame(rows_to_add,columns=cols)
new_df_30min = pd.concat([df_30min, rows_to_add_df], ignore_index=True)
print('30 done')
# df_1hr   #########################
last_1hour =df_1hr.loc[len(df_1hr)-1,'Unix Timestamp']
rows_to_add = combine_time_frames(new_df_1min, last_1hour, 60, "1 Hour")
rows_to_add_df = pd.DataFrame(rows_to_add,columns=cols)
new_df_1hr = pd.concat([df_1hr, rows_to_add_df], ignore_index=True)
print('1 done')
# df_4hr   #########################
last_4hour =df_4hr.loc[len(df_4hr)-1,'Unix Timestamp']
rows_to_add = combine_time_frames(new_df_1min, last_4hour, 240, "4 Hour")
rows_to_add_df = pd.DataFrame(rows_to_add,columns=cols)
new_df_4hr = pd.concat([df_4hr, rows_to_add_df], ignore_index=True)
print('4 done')
# df_12hr  #########################
last_12hour =df_12hr.loc[len(df_12hr)-1,'Unix Timestamp']
rows_to_add = combine_time_frames(new_df_1min, last_12hour, 720, "12 Hour")
rows_to_add_df = pd.DataFrame(rows_to_add,columns=cols)
new_df_12hr = pd.concat([df_12hr, rows_to_add_df], ignore_index=True)
print('12 done')
# df_24hr  #########################
last_24hour =df_24hr.loc[len(df_24hr)-1,'Unix Timestamp']
rows_to_add = combine_time_frames(new_df_1min, last_24hour, 1440, "24 Hour")
rows_to_add_df = pd.DataFrame(rows_to_add,columns=cols)
new_df_24hr = pd.concat([df_24hr, rows_to_add_df], ignore_index=True)
print('24 done')

5 done
30 done
1 done
4 done
12 done
24 done


In [215]:
# apply indicator functions 
def apply_indicators_to_whole(df):
    # SMAs #####################
    df['MA_5']  = df['Close'].rolling(window=5).mean()
    df['MA_8']  = df['Close'].rolling(window=8).mean()
    df['MA_10'] = df['Close'].rolling(window=10).mean()
    df['MA_13'] = df['Close'].rolling(window=13).mean()
    df['MA_20'] = df['Close'].rolling(window=20).mean()
    df['MA_50'] = df['Close'].rolling(window=50).mean()
    # RSI #####################
    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()
    rs = ema_up/ema_down
    df['RSI'] = 100 - (100/(1 + rs))
    # MacD #####################
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    macd = exp1 - exp2
    df['MACD']= macd
    exp3 = macd.ewm(span=9, adjust=False).mean()
    df['M_Signal']= exp3
    return df

new_df_1min  = apply_indicators_to_whole(new_df_1min)
new_df_5min  = apply_indicators_to_whole(new_df_5min)
new_df_30min = apply_indicators_to_whole(new_df_30min)
new_df_1hr   = apply_indicators_to_whole(new_df_1hr)
new_df_4hr   = apply_indicators_to_whole(new_df_4hr)
new_df_12hr  = apply_indicators_to_whole(new_df_12hr)
new_df_24hr  = apply_indicators_to_whole(new_df_24hr)


In [225]:
# printing to CSV's
new_df_1min.to_csv('BTC_Ticker_Data_1_Min.csv',index=False)
print('1 Min too CSV...')
new_df_5min.to_csv('BTC_Ticker_Data_5_Min.csv',index=False)
print('5 Min too CSV...')
new_df_30min.to_csv('BTC_Ticker_Data_30_Min.csv',index=False)
print('30 Min too CSV...')
new_df_1hr.to_csv('BTC_Ticker_Data_1_Hour.csv',index=False)
print('1 hour too CSV...')
new_df_4hr.to_csv('BTC_Ticker_Data_4_Hour.csv',index=False)
print('4 hour too CSV...')
new_df_12hr.to_csv('BTC_Ticker_Data_12_Hour.csv',index=False)
print('12 hour too CSV...')
new_df_24hr.to_csv('BTC_Ticker_Data_24_Hour.csv',index=False)
print('24 hour too CSV...')

1 Min too CSV...
5 Min too CSV...
30 Min too CSV...
1 hour too CSV...
4 hour too CSV...
12 hour too CSV...
24 hour too CSV...


TEST CODE BELOW THIS LINE 

In [216]:
new_df_5min.iloc[len(df_5min)-5:len(df_5min)+5]
new_df_5min

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444349040,2015-10-09 00:04:00,BTCUSD,243.95000,243.95000,243.95000,243.95000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,5 Min
1,1444349340,2015-10-09 00:09:00,BTCUSD,243.95000,243.95000,243.95000,243.95000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,5 Min
2,1444349640,2015-10-09 00:14:00,BTCUSD,243.95000,243.95000,243.95000,243.95000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,5 Min
3,1444349940,2015-10-09 00:19:00,BTCUSD,243.95000,243.95000,243.95000,243.95000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,5 Min
4,1444350240,2015-10-09 00:24:00,BTCUSD,243.95000,243.95000,243.95000,243.95000,0.00000,243.95000,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,5 Min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649670,1639250040,2021-12-11 19:14:00,BTCUSD,48787.20000,48791.80000,48767.70000,48787.40000,0.11043,48760.78000,48778.88750,48752.04000,48729.52308,48744.48500,48815.77800,50.69057,-15.70655,-22.10026,5 Min
649671,1639250340,2021-12-11 19:19:00,BTCUSD,48787.30000,48794.30000,48712.90000,48771.10000,5.10802,48749.18000,48775.76250,48769.21000,48735.62308,48739.13000,48820.82000,49.65859,-13.70832,-20.42187,5 Min
649672,1639250640,2021-12-11 19:24:00,BTCUSD,48771.10000,48788.60000,48750.20000,48780.80000,5.40588,48765.58000,48778.52500,48778.30000,48744.03077,48733.80000,48826.76000,50.30694,-11.21275,-18.58004,5 Min
649673,1639250940,2021-12-11 19:29:00,BTCUSD,48780.80000,48786.70000,48717.10000,48723.80000,2.75956,48770.08000,48759.95000,48771.07000,48753.54615,48733.49000,48830.51200,46.51580,-13.67675,-17.59939,5 Min


In [217]:
new_df_30min.iloc[len(df_30min)-5:len(df_30min)+5]
new_df_30min

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444350540,2015-10-09 00:29:00,BTCUSD,243.95000,243.95000,243.60000,243.60000,3.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,30 Min
1,1444352340,2015-10-09 00:59:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,30 Min
2,1444354140,2015-10-09 01:29:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,30 Min
3,1444355940,2015-10-09 01:59:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,30 Min
4,1444357740,2015-10-09 02:29:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,243.60000,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,30 Min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108274,1639243740,2021-12-11 17:29:00,BTCUSD,49015.40000,49031.50000,48800.00000,48948.40000,126.12432,48907.70000,48713.38750,48674.00000,48601.76154,48551.49000,48249.57000,59.41330,175.38112,116.84091,30 Min
108275,1639245540,2021-12-11 17:59:00,BTCUSD,48948.50000,48976.80000,48609.40000,48720.70000,41.22487,48936.04000,48757.71250,48691.20000,48637.13846,48583.92000,48272.53200,53.95593,163.39596,126.15192,30 Min
108276,1639247340,2021-12-11 18:29:00,BTCUSD,48720.70000,48854.60000,48588.90000,48599.40000,62.85802,48899.04000,48792.57500,48702.72000,48661.41538,48570.59000,48295.92600,51.25497,142.46746,129.41503,30 Min
108277,1639249140,2021-12-11 18:59:00,BTCUSD,48599.40000,48874.70000,48599.40000,48698.80000,20.64879,48796.54000,48819.67500,48735.99000,48673.76154,48586.21000,48315.30400,53.31724,132.37627,130.00728,30 Min


In [218]:
new_df_1hr.iloc[len(df_1hr)-5:len(df_1hr)+5]
new_df_1hr

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444352340,2015-10-09 00:59:00,BTCUSD,243.95000,243.95000,243.60000,243.60000,3.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,1 Hour
1,1444355940,2015-10-09 01:59:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,1 Hour
2,1444359540,2015-10-09 02:59:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,1 Hour
3,1444363140,2015-10-09 03:59:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,1 Hour
4,1444366740,2015-10-09 04:59:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,243.60000,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,1 Hour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54134,1639234740,2021-12-11 14:59:00,BTCUSD,48361.70000,48563.30000,48222.00000,48482.00000,49.40565,48419.68000,48390.47500,48345.34000,48354.55385,48180.68500,48212.70000,52.91428,46.48945,-8.44607,1 Hour
54135,1639238340,2021-12-11 15:59:00,BTCUSD,48482.10000,48784.50000,48455.10000,48784.40000,35.43479,48493.00000,48479.51250,48393.77000,48395.66154,48213.17500,48200.65800,56.93744,78.75572,8.99429,1 Hour
54136,1639241940,2021-12-11 16:59:00,BTCUSD,48788.80000,49284.80000,48730.60000,49015.40000,236.90887,48639.32000,48558.13750,48492.36000,48447.56923,48241.27500,48208.47400,59.76552,121.56540,31.50851,1 Hour
54137,1639245540,2021-12-11 17:59:00,BTCUSD,49015.40000,49031.50000,48609.40000,48720.70000,167.34918,48673.72000,48577.36250,48557.22000,48459.53077,48279.48500,48213.12000,54.81925,130.21156,51.24912,1 Hour


In [219]:
new_df_4hr.iloc[len(df_4hr)-5:len(df_4hr)+7]
new_df_4hr

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444363140,2015-10-09 03:59:00,BTCUSD,243.95000,243.95000,243.60000,243.60000,3.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,4 Hour
1,1444377540,2015-10-09 07:59:00,BTCUSD,243.60000,243.60000,243.60000,243.60000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,4 Hour
2,1444391940,2015-10-09 11:59:00,BTCUSD,243.60000,243.75000,243.60000,243.63000,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,0.00239,0.00048,4 Hour
3,1444406340,2015-10-09 15:59:00,BTCUSD,243.63000,244.00000,243.63000,244.00000,8.14508,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,0.03376,0.00713,4 Hour
4,1444420740,2015-10-09 19:59:00,BTCUSD,244.00000,244.06000,243.70000,244.06000,3.66263,243.77800,NaN,NaN,NaN,NaN,NaN,100.00000,0.06273,0.01825,4 Hour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13529,1639180740,2021-12-10 23:59:00,BTCUSD,48131.60000,48516.90000,46888.80000,47218.30000,449.04408,47938.76000,47921.05000,48106.41000,48554.21538,49190.66000,50367.07600,37.89404,-824.49433,-730.16956,4 Hour
13530,1639195140,2021-12-11 03:59:00,BTCUSD,47173.90000,48706.90000,46800.00000,48340.60000,503.12499,48069.88000,48005.01250,48019.74000,48386.94615,49060.40500,50200.51000,45.11767,-768.32533,-737.80072,4 Hour
13531,1639209540,2021-12-11 07:59:00,BTCUSD,48343.30000,48966.80000,47844.90000,48072.10000,261.12256,47943.10000,48066.32500,47978.11000,48271.39231,48931.50500,50031.42400,43.80497,-736.98125,-737.63682,4 Hour
13532,1639223940,2021-12-11 11:59:00,BTCUSD,48055.60000,48900.00000,47901.50000,48283.80000,218.77117,48009.88000,48048.78750,48039.60000,48135.43077,48838.09000,49869.62400,45.15978,-687.13751,-727.53696,4 Hour


In [220]:
new_df_12hr.iloc[len(df_12hr)-5:len(df_12hr)+5]
new_df_12hr

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444391940,2015-10-09 11:59:00,BTCUSD,243.95000,243.95000,243.60000,243.63000,4.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,12 Hour
1,1444435140,2015-10-09 23:59:00,BTCUSD,243.63000,249.97000,243.63000,245.39000,52.02784,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,0.14040,0.02808,12 Hour
2,1444478340,2015-10-10 11:59:00,BTCUSD,245.39000,245.54000,244.60000,244.60000,27.22370,NaN,NaN,NaN,NaN,NaN,NaN,96.66244,0.18578,0.05962,12 Hour
3,1444521540,2015-10-10 23:59:00,BTCUSD,244.60000,246.30000,244.60000,246.30000,12.12507,NaN,NaN,NaN,NaN,NaN,NaN,96.90205,0.35483,0.11866,12 Hour
4,1444564740,2015-10-11 11:59:00,BTCUSD,246.30000,246.30000,246.30000,246.30000,0.08100,245.24400,NaN,NaN,NaN,NaN,NaN,96.90205,0.48323,0.19157,12 Hour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4506,1639051140,2021-12-09 11:59:00,BTCUSD,50525.10000,50819.50000,48922.90000,49207.30000,1044.28051,50232.16000,49975.61250,49840.69000,50439.10000,52775.46000,56066.53000,38.56774,-2028.15874,-1980.97693,12 Hour
4507,1639094340,2021-12-09 23:59:00,BTCUSD,49207.40000,49645.90000,47298.40000,47581.60000,1647.21930,49438.74000,49742.90000,49673.34000,49710.63077,52263.03000,55708.26200,35.11562,-2137.37454,-2012.25646,12 Hour
4508,1639137540,2021-12-10 11:59:00,BTCUSD,47576.40000,48798.20000,47470.00000,48706.00000,838.60318,49049.92000,49751.58750,49609.25000,49328.12308,51814.38500,55367.43200,39.17105,-2108.88921,-2031.58301,12 Hour
4509,1639180740,2021-12-10 23:59:00,BTCUSD,48705.90000,50114.20000,46888.80000,47218.30000,2484.93858,48645.66000,49333.33750,49386.75000,49377.90769,51326.26500,55039.48200,35.96781,-2181.21564,-2061.50953,12 Hour


In [221]:
new_df_24hr.iloc[len(df_24hr)-5:len(df_24hr)+5]
new_df_24hr

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444435140,2015-10-09 23:59:00,BTCUSD,243.95000,249.97000,243.60000,245.39000,56.02784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,24 Hour
1,1444521540,2015-10-10 23:59:00,BTCUSD,245.39000,246.30000,244.60000,246.30000,39.34878,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,0.07259,0.01452,24 Hour
2,1444607940,2015-10-11 23:59:00,BTCUSD,246.30000,249.50000,245.96000,249.50000,14.48778,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,0.38391,0.08840,24 Hour
3,1444694340,2015-10-12 23:59:00,BTCUSD,249.50000,249.50000,247.60000,247.60000,62.78748,NaN,NaN,NaN,NaN,NaN,NaN,88.01748,0.47188,0.16509,24 Hour
4,1444780740,2015-10-13 23:59:00,BTCUSD,247.60000,252.87000,245.75000,250.42000,115.42576,247.84200,NaN,NaN,NaN,NaN,NaN,89.94355,0.76038,0.28415,24 Hour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2250,1638835140,2021-12-06 23:59:00,BTCUSD,49444.00000,50957.60000,47210.00000,50564.30000,4070.39292,51893.54000,53939.56250,54360.14000,54886.63846,56058.63500,60063.58400,35.41725,-2436.43597,-1676.68108,24 Hour
2251,1638921540,2021-12-07 23:59:00,BTCUSD,50544.80000,51955.00000,50100.00000,50650.10000,2712.74988,50718.28000,53042.05000,53946.66000,54382.54615,55572.75000,59835.83200,35.68319,-2529.95820,-1847.33650,24 Hour
2252,1639007940,2021-12-08 23:59:00,BTCUSD,50650.10000,51202.50000,48579.30000,50515.10000,2997.49648,50085.58000,52233.85000,53268.17000,53735.04615,55252.42000,59560.43600,35.43594,-2585.16840,-1994.90288,24 Hour
2253,1639094340,2021-12-09 23:59:00,BTCUSD,50525.10000,50819.50000,47298.40000,47581.60000,2691.49981,49750.88000,51026.81250,52243.31000,53257.55385,54726.52000,59191.35200,30.49179,-2832.97504,-2162.51731,24 Hour


In [ ]:
index_5 = df_5min.loc[len(df_5min)-1,'Unix Timestamp'] # get the last item in the df's unix value 
timeFrame = '5 Min'
rows_to_add = combine_time_frames(new_df_1min, index_5, 5, timeFrame)
cols = ['Unix Timestamp','Date','Symbol','Open','High','Low','Close','Volume','Time Frame']
rows_to_add_df = pd.DataFrame(rows_to_add,columns=cols)
new_df_5min = pd.concat([df_5min,rows_to_add_df], ignore_index=True)
new_df_5min

In [ ]:
# new_df_1min.iloc[3168006-10: 3168006+20]

In [ ]:
# used to get trade data but returns trades in groups of 1000
resp = requests.get('https://api.kraken.com/0/public/Trades?pair=xbtusd&since=1634429100').json()
resp['result']


In [ ]:
# used to get the ohlc data but only returns last 720 points of data 
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since=1634429100').json()
resp['result']


In [ ]:
#DO not change the below cell. will use this if something breaks and need a fresh copy of the code 

class KrakenWebSocketClient:
    """
    Kraken Client Interface

    """

    def __init__(self, ):
        websocket.enableTrace(False)
        ws = websocket.WebSocketApp("wss://ws.kraken.com/",
                                         on_message=self.on_message,
                                         on_error=self.on_error,
                                         on_close=self.on_close)
        self.ws = ws
        self.ws.on_open = self.on_open
        self.ws.run_forever()

    def on_message(self,ws, message):
        print(message)
        return message

    def on_error(self,ws, error):
        print(error)
        return error

    def on_close(self, a,b,c):
        self.ws.close()
        print("### closed ###")

    def run(self, *args):
        global driver
        driver = True
        while driver:
            try:
                time.sleep(1)
                
            except KeyboardInterrupt:
                driver = False
        time.sleep(1)
        self.ws.close()
        print("thread terminating...")

    def on_open(self,ws):
        self.ws.send('{"event":"subscribe", "subscription":{"name":"ohlc"}, "pair":["XBT/USD"]}')
        t = threading.Thread(target=self.run)
        t.start()
#         thread.start_new_thread(self.run, ())
        



In [ ]:
# use this if something reallly breaks.    but would rather use the class implementation 

def ws_thread(class_object):
    ws = websocket.WebSocketApp("wss://ws.kraken.com/", 
                                on_open    = ws_open, 
                                on_error   = ws_error,
                                on_message = partial(ws_message, class_object=class_object)) 
    ws.run_forever()
    
def ws_open(ws):
    ws.send('{"event":"subscribe", "subscription":{"name":"ohlc"}, "pair":["XBT/USD"]}')
def ws_error(ws, error):
    print(error)
def ws_message(ws, message, class_object):
    if "event" not in message: 
        formated_point = json.loads(message)
        print('WebSocket: '+formated_point[1][0]+' '+ formated_point[1][1]+' '+formated_point[1][2]+' '+str(formated_point[1][-1]))
        class_object.to_data_que(formated_point[1])
        

def main():
    BTC =BTC_Data()
    t = threading.Thread(target=ws_thread(BTC))
    t.start()


# if __name__ == '__main__':
#     main()

In [ ]:
# used to get the tradable pairs 
resp = requests.get('https://api.kraken.com/0/public/AssetPairs').json()

testing = pd.DataFrame(resp['result'])
# testing
for x in testing:
    print(testing[x].altname, testing[x].wsname)
# WBTC/USD